In [1]:
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Implementation of Multiclass Logistic Regression, allowing for $L_2$ regularization:

In [2]:
class MyLogisticRegression(BaseEstimator):

  def __init__(self, n_classes, lr, num_steps, C=1.0, penalty='l2', plot_training=False):
    self.n_classes = n_classes
    self.lr = lr
    self.num_steps = num_steps
    self.C = C
    self.penalty = penalty
    self.plot_training = plot_training
    
    self.W_best = None

    if penalty == 'l2':
      self.reg_func = self.ridge_reg
      self.grad_reg_func = self.grad_ridge_reg
    elif penalty == 'none':
      self.reg_func = lambda W: 0
      self.grad_reg_func = lambda W : 0

  
  def one_hot_encode(self, y):
    y_one_hot = np.zeros((len(y), self.n_classes), dtype=float)
    y_one_hot[np.arange(len(y)), y] = 1.
    return y_one_hot
  
  def softmax(self, X, W):
    exp_minus_XW = np.exp(-X @ W)
    return exp_minus_XW / exp_minus_XW.sum(axis=1)[:, None]
  
  def ridge_reg(self, W):
    return 1 / self.C * np.sum(W[1:] ** 2)
  
  def grad_ridge_reg(self, W):
    return 1 / self.C * 2 * np.r_[np.zeros(shape=(1, self.n_classes)), W[1:]] # we don't regularize the bias term(s)
  
  def fit(self, X, y):

    N, n_features = X.shape

    X_with_bias = np.c_[np.ones(shape=(N, 1)), X]
    W = np.random.rand(n_features + 1, self.n_classes)
    Y_oh = self.one_hot_encode(y)

    loss_list = []

    for i in tqdm(range(self.num_steps)):

      grad_loss = 1/N * X_with_bias.T @ (Y_oh - self.softmax(X_with_bias, W))
      W -= self.lr * (grad_loss + self.grad_reg_func(W))

      if self.plot_training:
        loss = 1/N * (np.trace(X_with_bias @ W @ Y_oh.T) + np.sum(np.log(np.exp(-X_with_bias @ W).sum(axis=1)))) + self.reg_func(W)
        loss_list.append(loss)

        clear_output(True)
        plt.plot(loss_list)
        plt.show()
    
    self.W_best = W
  
  def predict(self, X):
    X_with_bias = np.c_[np.ones(shape=(X.shape[0], 1)), X]
    probas = self.softmax(X_with_bias, self.W_best)
    return probas.argmax(axis=1)

We'll use the [digits dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#digits-dataset):

In [3]:
from sklearn.datasets import load_digits

In [4]:
X, y = load_digits(return_X_y=True)

In [5]:
X.shape, y.shape

((1797, 64), (1797,))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [7]:
(X_train.shape, X_test.shape), (y_train.shape, y_test.shape)

(((1257, 64), (540, 64)), ((1257,), (540,)))

# Training and comparison 

We'll compare our implementation of (Multiclass) Logistic Regression with the one from `sklearn`. This will be done:
- without regularization.
- with regularization, for different values of the hyperparameter `C`.

The chosen metric is *accuracy*.

For simplicity, we'll use *grid search* to compare the performance of both implementations with the corresponding set of hyperparameters:

In [8]:
param_grid = [
    
    {'C':[1.0], 'penalty': ['none']}, # without regularization
    {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'penalty': ['l2']} #with regularization
]

Let's try our implementation:

In [9]:
my_log_reg = MyLogisticRegression(n_classes=10, lr=1e-3, num_steps=5000, plot_training=False)

In [10]:
my_grid_search = GridSearchCV(my_log_reg, param_grid, cv=5, scoring='accuracy')

In [11]:
my_grid_search.fit(X_train, y_train)

100%|██████████| 5000/5000 [00:06<00:00, 762.82it/s]


GridSearchCV(cv=5,
             estimator=MyLogisticRegression(lr=0.001, n_classes=10,
                                            num_steps=5000),
             param_grid=[{'C': [1.0], 'penalty': ['none']},
                         {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'penalty': ['l2']}],
             scoring='accuracy')

Let's check the results:

In [12]:
import pandas as pd

In [13]:
my_log_reg_results = pd.DataFrame(my_grid_search.cv_results_)[['param_C', 'param_penalty', 'mean_test_score', 'rank_test_score']]
my_log_reg_results.rename(columns={'mean_test_score': 'mean_test_acc (my_log_reg)'}, inplace=True)
my_log_reg_results

,param_C,param_penalty,mean_test_acc (my_log_reg),rank_test_score
0,1,none,0.941128,4
1,0.1,l2,0.904537,6
2,0.5,l2,0.933978,5
3,1,l2,0.942721,3
4,1.5,l2,0.948289,2
5,2,l2,0.949877,1


Let's try the `sklearn` implementation:

In [14]:
skl_log_reg = LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [15]:
skl_grid_search = GridSearchCV(skl_log_reg, param_grid, cv=5, scoring='accuracy')

In [16]:
skl_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(multi_class='multinomial',
                                          solver='newton-cg'),
             param_grid=[{'C': [1.0], 'penalty': ['none']},
                         {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'penalty': ['l2']}],
             scoring='accuracy')

Let's check the results:

In [17]:
skl_log_reg_results = pd.DataFrame(skl_grid_search.cv_results_)[['param_C', 'param_penalty', 'mean_test_score', 'rank_test_score']]
skl_log_reg_results.rename(columns={'mean_test_score': 'mean_test_acc (skl_log_reg)'}, inplace=True)
skl_log_reg_results

,param_C,param_penalty,mean_test_acc (skl_log_reg),rank_test_score
0,1,none,0.950670,5
1,0.1,l2,0.956239,1
2,0.5,l2,0.950664,6
3,1,l2,0.952254,4
4,1.5,l2,0.953048,2
5,2,l2,0.953048,2


Let's compare the results of both implementations:

In [18]:
df_comparison = pd.concat([my_log_reg_results.drop('rank_test_score', axis=1), skl_log_reg_results[['mean_test_acc (skl_log_reg)']]], axis=1)
df_comparison['score diff (%)'] = np.abs(df_comparison.iloc[:, 2] - df_comparison.iloc[:, 3]) / df_comparison.iloc[:, 3] * 100
df_comparison

,param_C,param_penalty,mean_test_acc (my_log_reg),mean_test_acc (skl_log_reg),score diff (%)
0,1,none,0.941128,0.950670,1.003795
1,0.1,l2,0.904537,0.956239,5.406719
2,0.5,l2,0.933978,0.950664,1.755156
3,1,l2,0.942721,0.952254,1.001129
4,1.5,l2,0.948289,0.953048,0.499318
5,2,l2,0.949877,0.953048,0.332768


**<font color=blue>Conclusions:</font>**

- We see that the accuracy score for both implementations is approximately the same (within $2\%$ difference) for (almost) all the combinations of hyparameters `penalty` and `C`).
- There is **one case where the performance difference is above $2\%$** (case `C=0.1` with a difference of $5.9\%$). We can't say for sure why that happened, unless we run both algorithms multiple times. Even so, due the direct proportionality of `mean_test_acc (my_log_reg)` with `C`, it seems that a stronger regularization leads to a slower convergence of the gradient descent used in the custom implementation:
  - On the one hand, the `sklearn` implementation applies a second-order method for optimization (`newton-cg`), while the custom implementation uses a first-order one (plain Gradient Descent). As we know second-order methods, although computationally time-consuming, have a faster convergence to the optimum in comparison with first-order methods, which converge slower but are less computationally expensive.
  - On the other hand, the `sklearn` implementation uses a tolerance criteria (`tol=1e-4`) as the optimizer's stopping condition instead of the number of iterations as in the custom implementation (`num_steps`). **This means that the convergence for our implementation is not only slower, but it also stops before getting close enough to the optimum.**